<a href="https://colab.research.google.com/github/sofi388/nlp_ucph/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 36

1. Explore the dataset from https://huggingface.co/datasets/coastalcph/tydi_xor_rc. Familiarize yourself with the dataset card, download the
dataset and explore its columns. Summarize basic data statistics for training and validation data in each of the languages Finnish (fi), Japanese
(ja) and Russian (ru). # Sofia + Tom

2. For each of the languages Finnish, Japanese and Russian, report the 5
most common words in the questions from the training set. What kind of
words are they?

3. Implement a rule-based classifier that predicts whether a question is answerable or impossible, only using the document (context) and question.
You may use machine translation as a component. Use the answerable
field to evaluate it on the validation set. What is the performance of your
classifier for each of the languages Finnish, Japanese and Russian?



####  1. Exploring the dataset

In [5]:
import pandas as pd

splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])

In [ ]:
print(df.head)

<bound method NDFrame.head of                                                 question  \
0      উইকিলিকস কত সালে সর্বপ্রথম ইন্টারনেটে প্রথম তথ...   
1               দ্বিতীয় বিশ্বযুদ্ধে কোন দেশ পরাজিত হয় ?   
2      মার্কিন যুক্তরাষ্ট্রের সংবিধান অনুযায়ী মার্কিন...   
3      আরব-ইসরায়েলি যুদ্ধে আরবের মোট কয়জন সৈন্যের মৃ...   
4              বিশ্বে প্রথম পুঁজিবাদী সমাজ কবে গড়ে ওঠে ?   
...                                                  ...   
15321              కోళ్లు ఎక్కువగా ఏ దేశంలో కనిపిస్తాయి?   
15322       క్షయ వ్యాధికి విరుగుడు ఏ దేశంలో కనుగొన్నారు?   
15323                 ఖురాన్ ఏ అరబ్బీ భాషలో ఎవరు రాసారు?   
15324  టెక్సస్ రాష్ట్రంలోని అతిపెద్ద మానవ నిర్మితం ఏది ?   
15325         తమిళనాడులో రాష్ట్ర మొదటి ముఖ్యమంత్రి ఎవరు?   

                                                 context lang  answerable  \
0      WikiLeaks () is an international non-profit or...   bn        True   
1      The war in Europe concluded with an invasion o...   bn        True   
2      Same-sex ma

In [ ]:
df.describe()

,answer_start
count,15326.000000
mean,157.062769
std,226.748482
min,-1.000000
25%,13.000000
50%,78.000000
75%,210.000000
max,3964.000000


In [6]:
# Train and validation sets
train_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
val_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

# Explore the dataset columns (training and validation)
print("Training Data Columns:", train_df.columns)
print("Validation Data Columns:", val_df.columns)

Training Data Columns: Index(['question', 'context', 'lang', 'answerable', 'answer_start', 'answer',
       'answer_inlang'],
      dtype='object')
Validation Data Columns: Index(['question', 'context', 'lang', 'answerable', 'answer_start', 'answer',
       'answer_inlang'],
      dtype='object')


In [ ]:
finnish_questions = df[df['lang'] == 'fi']
japanese_questions = df[df['lang'] == 'ja']
russian_questions = df[df['lang'] == 'ru']

# Summarize basic statistics
print("Statistics for Finnish Questions:")
print(finnish_questions.describe(include='all'))  # or include only numeric features by default
print("\nStatistics for Japanese Questions:")
print(japanese_questions.describe(include='all'))
print("\nStatistics for Russian Questions:")
print(russian_questions.describe(include='all'))

Statistics for Finnish Questions:
                                                question  \
count                                               2126   
unique                                              2100   
top     Kuinka monta peliä Final Fantasy-sarjaan kuuluu?   
freq                                                   3   
mean                                                 NaN   
std                                                  NaN   
min                                                  NaN   
25%                                                  NaN   
50%                                                  NaN   
75%                                                  NaN   
max                                                  NaN   

                                                  context  lang answerable  \
count                                                2126  2126       2126   
unique                                               2054     1          2   
top     Wes

We can see, that there are 2301 Japanese questions (2100 unique), 2126 Finnish questions (1743 unique) and 1983 Russian questions (1612 unique).

#### 2.  5 most common words

In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
from collections import Counter
from googletrans import Translator
translator = Translator()

In [ ]:
# Find 5 most common words in Finnish in df dataset

finnish_questions = df[df['lang'] == 'fi']['question']

# Combine all questions into a single string
all_questions_text = ' '.join(finnish_questions.astype(str))

# Tokenize the text
words = all_questions_text.lower().split()

# Count the frequency of each word
word_counts = Counter(words)

# Get the 5 most common words
top_5_words = word_counts.most_common(5)

# Translate words dynamically
translated_top_5 = []
for word, count in top_5_words:
    translation = translator.translate(word, src='fi', dest='en').text  # Translate word from Finnish to English
    translated_top_5.append((word, count, translation))

# Print the top 5 words with translations
print("Top 5 most common words in Finnish questions with English translations:")
for word, count, translation in translated_top_5:
    print(f"{word} (count: {count}) - English: {translation}")

# print("Top 5 most common words in Finnish questions:", top_5_words)


Top 5 most common words in Finnish questions with English translations:
on (count: 582) - English: there is
mikä (count: 328) - English: What
milloin (count: 287) - English: When
vuonna (count: 227) - English: in
kuka (count: 215) - English: Who


In [ ]:
# Find 5 most common words in Russian in df dataset
russian_questions = df[df['lang'] == 'ru']['question']

all_questions_text = ' '.join(russian_questions.astype(str))

words = all_questions_text.lower().split()

word_counts = Counter(words)

top_5_words = word_counts.most_common(5)

# Translate words dynamically
translated_top_5 = []
for word, count in top_5_words:
    translation = translator.translate(word, src='ru', dest='en').text  # Translate word from Russian to English
    translated_top_5.append((word, count, translation))

# Print the top 5 words with translations
print("Top 5 most common words in Russian questions with English translations:")
for word, count, translation in translated_top_5:
    print(f"{word} (count: {count}) - English: {translation}")

#print("Top 5 most common words in Russian questions:", top_5_words)




Top 5 most common words in Russian questions with English translations:
в (count: 978) - English: V
сколько (count: 426) - English: How many
на (count: 385) - English: on
когда (count: 251) - English: When
кто (count: 209) - English: Who


For both Finnish and Russian it appears the the most common words are question
words or "connecting"/grammar words.

Let's proceed to Japanese:

In [ ]:
# Find 5 most common words in Japanese in df dataset
japanese_questions = df[df['lang'] == 'ja']['question']

all_questions_text = ' '.join(japanese_questions.astype(str))

words = all_questions_text.lower().split()

word_counts = Counter(words)

top_5_words = word_counts.most_common(5)

print("Top 5 most common words in Japanese questions:", top_5_words)

Top 5 most common words in Japanese questions: [('pygが最初に発表した曲は何', 3), ('オーロラ号の漂流時、オーロラ号に乗組員は何人いた？', 3), ('カンザスシティが設立されたのはいつ', 3), ('クロイツ群が初めて発見されたのはいつ', 3), ('ライド', 2)]


We can see that instead of actually counting words in japanese, it counted whole sentences. We'll try to use a specific tokenizer for japanese:

In [ ]:
# Load tokenizer directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_ja = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ja-en")
#model_ja = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ja-en")

tokenizer_fi = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fi-en")
#model_fi = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-fi-en")

tokenizer_ru = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
#model_ru = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")



/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/832k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

In [ ]:
def tokenize_japanese_sentences(sentence):
    tokens = tokenizer_ja.tokenize(sentence)
    return " ".join(tokens)

japanese_questions = df[df['lang'] == 'ja']['question']

# Apply tokenization to the 'japanese_questions' series
tokenized_japanese_questions = japanese_questions.apply(tokenize_japanese_sentences)

# Print the original and tokenized questions
for original, tokenized in zip(japanese_questions.head(5), tokenized_japanese_questions.head(5)):
    print(f"Original: {original}")
    print(f"Tokenized: {tokenized}")
    print("-" * 50)

Original: ポーランドで農地改革が行われたことがある？
Tokenized: ▁ポーランド で 農 地 改革 が 行わ れた こと がある ?
--------------------------------------------------
Original: ビスカヤ県で初めて進出した大規模鉱業会社は何？
Tokenized: ▁ビ スカ ヤ 県 で 初めて 進 出 した 大 規模 鉱 業 会社 は 何 ?
--------------------------------------------------
Original: 古代ローマ帝国はいつ起きた？
Tokenized: ▁古代 ローマ 帝国 は いつ 起き た ?
--------------------------------------------------
Original: スペイン・バスク州の州都はどこですか？
Tokenized: ▁スペイン ・ バス ク 州 の 州 都 は どこ です か ?
--------------------------------------------------
Original: イタリア王国海軍は第一次世界大戦中に何隻戦艦をつくった？
Tokenized: ▁イタリア 王国 海軍 は 第一 次 世界 大戦 中 に 何 隻 戦艦 を つく った ?
--------------------------------------------------


Now that the Japanese questions are properly tokenized, we can proceed to find the most common words:

In [ ]:
import re

def clean_text(text):
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)  # Removes non-word characters (punctuation)
    return text.lower().split()

# Convert tokenized questions into a single string
all_questions_text = ' '.join(tokenized_japanese_questions.astype(str))

# Clean the text by removing punctuation
cleaned_words = clean_text(all_questions_text)

# Count word frequencies
word_counts = Counter(cleaned_words)

# Get the top 5 most common words
top_5_words = word_counts.most_common(5)

# Translate words dynamically
translated_top_5 = []
for word, count in top_5_words:
    translation = translator.translate(word, src='ja', dest='en').text  # Translate word from Japanese to English
    translated_top_5.append((word, count, translation))

# Print the top 5 words with translations
print("Top 5 most common words in Japanese questions with English translations:")
for word, count, translation in translated_top_5:
    print(f"{word} (count: {count}) - English: {translation}")

Top 5 most common words in Japanese questions with English translations:
は (count: 2224) - English: teeth
の (count: 1632) - English: of
何 (count: 542) - English: what
した (count: 469) - English: did
いつ (count: 459) - English: when


As we can see, the filtered Japanese results are similar to the Finnish and Russian results.

Note the "は" token. Altough it can be translated to "teeth", it is most commonly used as grammatical particle - used to indicate the subject of the sentence. It is a frequently used particle which explains why we see it so much.

In [ ]:
def clean_text(text):
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)  # Removes non-word characters (punctuation)
    return text.lower().split()

def find_most_common_words(tokenizer, language, num_words=5):
    # Filter questions for the specified language
    questions_ = df[df['lang'] == language]['question']

    # Tokenize questions
    tokenized_questions = questions_.apply(lambda x: " ".join(tokenizer.tokenize(x)))

    # Convert tokenized questions into a single string
    all_questions_text = ' '.join(tokenized_questions.astype(str))

    # Clean the text by removing punctuation
    cleaned_words = clean_text(all_questions_text)

    # Count word frequencies
    word_counts = Counter(cleaned_words)

    # Get the top n most common words
    top_n_words = word_counts.most_common(num_words)

    # Translate words dynamically
    translated_top_n = []
    for word, count in top_n_words:
        translation = translator.translate(word, src=language, dest='en').text  # Translate word to English
        translated_top_n.append((word, count, translation))

    # Print the top n words with translations
    for word, count, translation in translated_top_n:
        print(f"{word} (count: {count}) - English: {translation}")

print("Top 5 most common words in Finnish questions:")
find_most_common_words(tokenizer_fi, 'fi')

print("\nTop 5 most common words in Japanese questions:")
find_most_common_words(tokenizer_ja, 'ja')

print("\nTop 5 most common words in Russian questions:")
find_most_common_words(tokenizer_ru, 'ru')

Top 5 most common words in Finnish questions:
on (count: 678) - English: there is
n (count: 420) - English: of
mikä (count: 328) - English: What
milloin (count: 287) - English: When
a (count: 242) - English: a

Top 5 most common words in Japanese questions:
は (count: 2224) - English: teeth
の (count: 1632) - English: of
何 (count: 542) - English: what
した (count: 469) - English: did
いつ (count: 459) - English: when

Top 5 most common words in Russian questions:
в (count: 1172) - English: V
на (count: 493) - English: on
а (count: 451) - English: A
сколько (count: 426) - English: How many
е (count: 317) - English: e


####  3. Rule-based Classifier

Classifier that predicts whether a question is answerable or impossible, only using the document (context) and question. You may use machine translation as a component.

In [ ]:
!pip install tqdm

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics import accuracy_score, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm  # Import tqdm for progress bars

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def clean_tokens(tokens):
    # Remove punctuation tokens and stopwords, and change words to "basic" form (lemmatize)
    cleaned_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]
    return cleaned_tokens

def predict_answerability(context, question, lang):
    translated_question = translator.translate(question, src=lang, dest='en').text

    # Tokenize both the context and the translated question
    tokenized_context = tokenizer.tokenize(context.lower())
    tokenized_question = tokenizer.tokenize(translated_question.lower())

    # Clean tokens of common words and punctuation
    cleaned_context = clean_tokens(tokenized_context)
    cleaned_question = clean_tokens(tokenized_question)

    # Check for overlap
    for token in cleaned_question:
        if token in cleaned_context:
            return 'True', translated_question

    # If there is no overlap, we determine the question unanswerable
    return 'False', translated_question


# Evaluate the classifier on the validation set
def evalute_classifier(df_, num_samples=100, seed=420):
  predictions = []
  true_labels = []

  sampled_rows = df_.sample(n=num_samples, random_state=seed)

  for index, row in tqdm(sampled_rows.iterrows(), total=num_samples):
      context = row['context']  # Always in English
      question = row['question']  # Could be in any language
      lang = row['lang']

      prediction, translated_question = predict_answerability(context, question, lang)

      predictions.append(1 if prediction == 'True' else 0)
      true_labels.append(1 if str(row['answerable']) == 'True' else 0)

      """print(f"Context: {context}")
      print(f"Question: {question}")
      print(f"Translated Question: {translated_question}")
      print(f"Prediction: {prediction}")  # This will be 'answerable' or 'impossible'
      print(f"True Label: {row['answerable']}")
      print("-" * 50)"""

  # Calculate accuracy
  accuracy = accuracy_score(true_labels, predictions)
  print(f"Accuracy: {accuracy * 100:.2f}%")

  # Calculate FP and FN percentages
  cm = confusion_matrix(true_labels, predictions, labels=[1, 0])
  tn, fp, fn, tp = cm.ravel()
  false_positive_percentage = (fp / (fp + tn)) * 100 if (fp + tn) > 0 else 0
  false_negative_percentage = (fn / (fn + tp)) * 100 if (fn + tp) > 0 else 0

  print(f"False Positive Percentage: {false_positive_percentage:.2f}%")
  print(f"False Negative Percentage: {false_negative_percentage:.2f}%")


print(f"Classifer evaluation for Finnish:")
evalute_classifier(val_df[val_df['lang'] == 'fi'])
print("-" * 50)
print(f"Classifer evaluation for Japanese:")
evalute_classifier(val_df[val_df['lang'] == 'ja'])
print("-" * 50)
print(f"Classifer evaluation for Russian:")
evalute_classifier(val_df[val_df['lang'] == 'ru'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Classifer evaluation for Finnish:


100%|██████████| 100/100 [00:11<00:00,  8.74it/s]


Accuracy: 72.00%
False Positive Percentage: 1.41%
False Negative Percentage: 93.10%
--------------------------------------------------
Classifer evaluation for Japanese:


100%|██████████| 100/100 [00:07<00:00, 13.17it/s]


Accuracy: 66.00%
False Positive Percentage: 13.43%
False Negative Percentage: 75.76%
--------------------------------------------------
Classifer evaluation for Russian:


100%|██████████| 100/100 [00:08<00:00, 11.59it/s]

Accuracy: 72.00%
False Positive Percentage: 9.59%
False Negative Percentage: 77.78%


In [ ]:
# Just some attempts to connect the project with the gitub repo


#!git clone https://github.com/sofi388/nlp_ucph.git

In [ ]:
#%cd nlp_ucph

In [ ]:
#%cd /content/nlp_ucph

In [ ]:
#!find /content -name "NLP_Project.ipynb"

# Week 37

Let k be the number of members in your group (k ∈ {1, **2**, 3}). Implement
k different * language models for the questions in the three languages Finnish,
Japanese and Russian, as well as for the document contexts in English (total
**2 × 4** language models), using the training data. Evaluate each of them on the
validation data, report their performance and discuss the results. Reminder: a
language model is a function that takes text as input and returns its probability.

\* Different approach (n-gram/neural) or different n, different smoothing etc.
2


In [ ]:
import pandas as pd

splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])

train_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
val_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

**How to implement N-gram model:**

Step 1: Tokenization
Tokenize your text data into words or characters, depending on the N-gram size you choose (e.g., bigrams, trigrams).

Step 2: Build N-gram Frequency Tables
Create N-grams: Extract N-grams from the tokenized text.

For bigrams, create pairs of consecutive tokens.
For trigrams, create triplets of consecutive tokens.
Count Frequencies: Count the occurrences of each N-gram in your dataset.

Step 3: Compute Probabilities
Calculate Probabilities: Compute the probability of each N-gram by dividing its frequency by the frequency of its prefix (for bigrams) or the frequency of its prefix (for trigrams).

Step 4: Generate Text
Text Generation: Use the computed probabilities to generate text. Start with a seed word and iteratively add the most likely subsequent word based on the N-gram probabilities.

Repeat the above process for each language: Finnish, Japanese, Russian.

Tokenize the text using language-specific tools if necessary (see week 36).
Build N-gram models and compute probabilities.

 ####  1. Trigram model (Sofia)

In [11]:
from collections import defaultdict, Counter
import math

class TrigramModel:
    def __init__(self, tokenizer, smoothing_coef):
        # Initialize the trigram, bigram, and unigram counts
        self.trigram_counts = defaultdict(Counter)
        self.bigram_counts = defaultdict(Counter)
        self.unigram_counts = Counter()
        self.known_words = set()
        self.tokenizer = tokenizer
        self.smoothing_coef = smoothing_coef

    def train(self, sentences):
        """
        Train the trigram model on a list of tokenized sentences.
        :param sentences: List of sentences where each sentence is a list of words.
        """
        tokenized_sentences = [self.tokenizer.tokenize(sentence.lower()) for sentence in sentences]

        for sentence in tokenized_sentences:
            # Replace first occurrences with 'OOV' and update known_words set
            for i in range(len(sentence)):
                word = sentence[i]
                if word not in self.known_words:
                    sentence[i] = 'OOV'
                    self.known_words.add(word)

            # Count unigrams, bigrams, and trigrams
            for i in range(len(sentence) - 2):
                self.unigram_counts[sentence[i]] += 1
                self.bigram_counts[sentence[i]][sentence[i + 1]] += 1
                self.trigram_counts[(sentence[i], sentence[i + 1])][sentence[i + 2]] += 1

            # Count remaining bigram and unigram
            if len(sentence) > 1:
                self.unigram_counts[sentence[-2]] += 1
                self.bigram_counts[sentence[-2]][sentence[-1]] += 1
            self.unigram_counts[sentence[-1]] += 1  # Count the last word as a unigram

    def unigram_probability(self, word):
        """
        Calculate the unigram probability P(word).
        :param word: The word for which to calculate the probability.
        :return: The probability of the word.
        """
        total_count = sum(self.unigram_counts.values())
        vocab_size = len(self.known_words)  # Unique words
        if word in self.unigram_counts:
            return self.unigram_counts[word] / total_count
        else:
            return self.unigram_counts['OOV'] / total_count if 'OOV' in self.unigram_counts else 0

    def bigram_probability(self, w1, w2):
        """
        Calculate the bigram probability P(w2 | w1).
        :param w1: Previous word.
        :param w2: Current word.
        :return: Probability of w2 given w1.
        """
        vocab_size = len(self.known_words)
        if self.unigram_counts[w1] > 0:
            return (self.bigram_counts[w1][w2] + self.smoothing_coef) / (self.unigram_counts[w1] + self.smoothing_coef * vocab_size)
        else:
            return (self.bigram_counts['OOV'][w2] + self.smoothing_coef) / (self.unigram_counts['OOV'] + self.smoothing_coef * vocab_size)

    def trigram_probability(self, w1, w2, w3):
        """
        Calculate the trigram probability P(w3 | w1, w2).
        :param w1: First word.
        :param w2: Second word.
        :param w3: Third word.
        :return: Probability of w3 given w1 and w2.
        """
        vocab_size = len(self.known_words)

        # Get the bigram count for (w1, w2)
        bigram_count = self.bigram_counts[w1][w2]

        if bigram_count > 0:
            return (self.trigram_counts[(w1, w2)][w3] + self.smoothing_coef) / (bigram_count + self.smoothing_coef * vocab_size)
        else:
            # Handle the case where (w1, w2) is unknown, fallback to a smoothed probability for OOV
            return (self.trigram_counts[('OOV', 'OOV')][w3] + self.smoothing_coef) / (self.bigram_counts['OOV']['OOV'] + self.smoothing_coef * vocab_size)

    def replace_unknowns(self, sentence):
        """
        Replace unknown words in the sentence with 'OOV'.
        :param sentence: A list of words (tokens).
        :return: List of words with unknown words replaced by 'OOV'.
        """
        return ['OOV' if word not in self.known_words else word for word in sentence]

    def sentence_perplexity(self, sentence):
        """
        Calculate the perplexity of a sentence.
        :param sentence: A sentence string.
        :return: The perplexity of the sentence.
        """
        tokens = self.tokenizer.tokenize(sentence.lower())
        tokens = self.replace_unknowns(tokens)

        T = len(tokens)

        if T == 0:
            return float('inf')

        # Initial unigram and bigram probabilities for first two tokens
        prob = math.pow(self.unigram_probability(tokens[0]), 1/T)
        if T > 1:
            prob *= math.pow(self.bigram_probability(tokens[0], tokens[1]), 1/T)

        # Trigram probabilities for the rest of the tokens
        for i in range(2, T):
            prob *= math.pow(self.trigram_probability(tokens[i - 2], tokens[i - 1], tokens[i]), 1/T)

        return 1 / prob if prob > 0 else float('inf')


In [2]:
!pip install transformers

In [3]:
from transformers import AutoTokenizer
# Load the XLM-Roberta tokenizer
tokenizer_multilingual = AutoTokenizer.from_pretrained("xlm-roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
def evaluate_model(model, validation_sentences):
    """
    Evaluate the model by calculating the perplexity on validation sentences.
    :param model: The TrigramModel instance to evaluate.
    :param validation_sentences: List of validation sentences.
    :return: Average perplexity of the model on the validation set.
    """
    total_perplexity = 0
    count = 0

    for sentence in validation_sentences:
        perplexity = model.sentence_perplexity(sentence)
        total_perplexity += perplexity
        count += 1

    return total_perplexity / count if count > 0 else float('inf')

smoothing_coef = 0.1 # Random choice. Not possible to calibrate because we're using the validation set as a testing set.

Finnish_model = TrigramModel(tokenizer_multilingual, smoothing_coef)
Finnish_model.train(train_df[train_df['lang'] == 'fi']['question'])

Japanese_model = TrigramModel(tokenizer_multilingual, smoothing_coef)
Japanese_model.train(train_df[train_df['lang'] == 'ja']['question'])

Russian_model = TrigramModel(tokenizer_multilingual, smoothing_coef)
Russian_model.train(train_df[train_df['lang'] == 'ru']['question'])

Context_model = TrigramModel(tokenizer_multilingual, smoothing_coef)
Context_model.train(train_df['context'])

validation_fi = val_df[val_df['lang'] == 'fi']['question']
validation_ja = val_df[val_df['lang'] == 'ja']['question']
validation_ru = val_df[val_df['lang'] == 'ru']['question']
validation_context = val_df['context']


# Evaluate models
finnish_perplexity = evaluate_model(Finnish_model, validation_fi)
print(f"Finnish Model Perplexity: {finnish_perplexity:.2f}")

japanese_perplexity = evaluate_model(Japanese_model, validation_ja)
print(f"Japanese Model Perplexity: {japanese_perplexity:.2f}")

russian_perplexity = evaluate_model(Russian_model, validation_ru)
print(f"Russian Model Perplexity: {russian_perplexity:.2f}")

context_perplexity = evaluate_model(Context_model, validation_context)
print(f"Context Model Perplexity: {context_perplexity:.2f}")


Finnish Model Perplexity: 723.31
Japanese Model Perplexity: 481.33
Russian Model Perplexity: 689.54
Context Model Perplexity: 2265.00


 ####  2. Bigram model (Tom)

The dataset for each langauge is rather small. In order to avoid overfitting I chose to implement a simple model - the bigram one.

In [13]:
from collections import defaultdict, Counter
import math

class BigramModel:
    def __init__(self, tokenizer, smoothing_coef):
        # Initialize the bigram and unigram counts
        self.bigram_counts = defaultdict(Counter)
        self.unigram_counts = Counter()
        self.known_words = set()
        self.tokenizer = tokenizer
        self.smoothing_coef = smoothing_coef

    def train(self, sentences):
        """
        Train the bigram model on a list of tokenized sentences.
        :param sentences: List of sentences where each sentence is a list of words.
        """
        tokenized_sentences = [self.tokenizer.tokenize(sentence.lower()) for sentence in sentences]

        for sentence in tokenized_sentences:
            # Replace first occurrences with 'OOV' and update known_words set
            for i in range(len(sentence)):
                word = sentence[i]
                if word not in self.known_words:
                    sentence[i] = 'OOV'
                    self.known_words.add(word)

            # Count bigrams and unigrams
            for i in range(len(sentence) - 1):
                self.unigram_counts[sentence[i]] += 1
                self.bigram_counts[sentence[i]][sentence[i + 1]] += 1
            self.unigram_counts[sentence[-1]] += 1  # Count the last word as a unigram

    def unigram_probability(self, word):
        """
        Calculate the unigram probability P(word).
        :param word: The word for which to calculate the probability.
        :return: The probability of the word.
        """
        total_count = sum(self.unigram_counts.values())
        vocab_size = len(self.known_words)  # Unique words
        if word in self.unigram_counts:
            return self.unigram_counts[word] / total_count
        else:
            return self.unigram_counts['OOV'] / total_count if 'OOV' in self.unigram_counts else 0

    def bigram_probability(self, w1, w2):
        """
        Calculate the bigram probability P(w2 | w1).
        :param w1: Previous word.
        :param w2: Current word.
        :return: Probability of w2 given w1.
        """
        vocab_size = len(self.known_words)  # Unique words
        if self.unigram_counts[w1] > 0:
            return (self.bigram_counts[w1][w2] + self.smoothing_coef) / (self.unigram_counts[w1] + self.smoothing_coef * vocab_size)
        else:
            return (self.bigram_counts['OOV'][w2] + self.smoothing_coef) / (self.unigram_counts['OOV'] + self.smoothing_coef * vocab_size)

    def replace_unknowns(self, sentence):
        """
        Replace unknown words in the sentence with 'OOV'.
        :param sentence: A list of words (tokens).
        :return: List of words with unknown words replaced by 'OOV'.
        """
        return ['OOV' if word not in self.known_words else word for word in sentence]

    def sentence_preplexity(self, sentence):
        """
        Calculate the perplexity of a sentence.
        :param sentence: A sentence string.
        :return: The perplexity of the sentence.
        """
        tokens = self.tokenizer.tokenize(sentence.lower())
        tokens = self.replace_unknowns(tokens)

        T = len(tokens)

        if T == 0:
          return float('inf')

        prob = math.pow(self.unigram_probability(tokens[0]), 1/T)
        for i in range(T - 1):
          prob *= math.pow(self.bigram_probability(tokens[i], tokens[i + 1]), 1/T)

        if prob > 0:
          return 1 / prob  # Perplexity formula
        else:
          #print unigram probabilty for first token
          return float('inf')
        return prob

Now that we have a working model template, we can apply it to the Russian / Japanese / Finnish questions and the English context.

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer
# Load the XLM-Roberta tokenizer
tokenizer_multilingual = AutoTokenizer.from_pretrained("xlm-roberta-base")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
def evaluate_model(model, validation_sentences):
    """
    Evaluate the model by calculating the perplexity on validation sentences.
    :param model: The BigramModel instance to evaluate.
    :param validation_sentences: List of validation sentences.
    :return: Average perplexity of the model on the validation set.
    """
    total_perplexity = 0
    count = 0

    for sentence in validation_sentences:
        perplexity = model.sentence_preplexity(sentence)
        total_perplexity += perplexity
        count += 1

    return total_perplexity / count if count > 0 else float('inf')

smoothing_coef = 0.1 # Random choice. Not possible to calibrate because we're using the validation set as a testing set.

Finnish_model = BigramModel(tokenizer_multilingual, smoothing_coef)
Finnish_model.train(train_df[train_df['lang'] == 'fi']['question'])

Japanese_model = BigramModel(tokenizer_multilingual, smoothing_coef)
Japanese_model.train(train_df[train_df['lang'] == 'ja']['question'])

Russian_model = BigramModel(tokenizer_multilingual, smoothing_coef)
Russian_model.train(train_df[train_df['lang'] == 'ru']['question'])

Context_model = BigramModel(tokenizer_multilingual, smoothing_coef)
Context_model.train(train_df['context'])

validation_fi = val_df[val_df['lang'] == 'fi']['question']
validation_ja = val_df[val_df['lang'] == 'ja']['question']
validation_ru = val_df[val_df['lang'] == 'ru']['question']
validation_context = val_df['context']


# Evaluate models
finnish_perplexity = evaluate_model(Finnish_model, validation_fi)
print(f"Finnish Model Perplexity: {finnish_perplexity:.2f}")

japanese_perplexity = evaluate_model(Japanese_model, validation_ja)
print(f"Japanese Model Perplexity: {japanese_perplexity:.2f}")

russian_perplexity = evaluate_model(Russian_model, validation_ru)
print(f"Russian Model Perplexity: {russian_perplexity:.2f}")

context_perplexity = evaluate_model(Context_model, validation_context)
print(f"Context Model Perplexity: {context_perplexity:.2f}")


Finnish Model Perplexity: 575.76
Japanese Model Perplexity: 290.49
Russian Model Perplexity: 567.91
Context Model Perplexity: 556.19


To check: Why the perplexity of Trigram is higher than the bigram? Shouldn't it be the opposite?... (sofia)

# Week 38


Let k be the number of members in your group. For each of the three languages
Finnish, Japanese and Russian separately, using the training data, train k different classifiers that receive the document (context) and question as input
and predict whether the question is answerable or impossible given the context.

Evaluate the classifiers on the respective validation sets, report and analyse the performance for each language and compare the scores across languages.
The classifiers can use machine translation, linguistic/lexical features (e.g.,
bag-of-words, n-gram counts, word overlap) word embeddings, or word/sentence
representations from (multilingual) neural language models.8 You can also train
or fine-tune your own neural language models on the dataset. Different from
1(c), however, they must be learned rather than rule-based. Motivate your
choice of features and classifier

1. Classifier 1
2. Classifier 2